<a href="https://colab.research.google.com/github/MuhammadIrzam447/MultiModel/blob/master/Valid_62.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !gdown https://drive.google.com/uc?id=1ytJdl1n4e2Y-F8a8mJab0S_4TgyrkE-4

In [ ]:
# !unzip "/content/joint_representations.zip"

In [ ]:
# !gdown https://drive.google.com/uc?id=1wgl3QGXZ4m2aLg3T-1TDXQqSP31RuXgL

In [ ]:
# !unzip "/content/hateful_train+test_unseen.zip"

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# Load Model


In [1]:
!pip install transformers evaluate datasets
import requests
import torch
from PIL import Image
from transformers import *
from tqdm import tqdm

/usr/local/lib/python3.10/dist-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation_tf_utils.py:24: FutureWarning: Importing `TFGenerationMixin` from `src/transformers/generation_tf_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import TFGenerationMixin` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation_flax_utils.py:24: FutureWarning: Importing `FlaxGenerationMixin` from `src/transformers/generation_flax_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import FlaxGenerationMixin` instead.
  warnings.warn(
Xformers is not installed correctly. If you want to use memory_efficient_attentio

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [3]:
model = ViTForImageClassification.from_pretrained(f"/content/Model/Models-Train-14/checkpoint-63000").to(device)
image_processor = ViTImageProcessor.from_pretrained(f"/content/Model/Models-Train-14/checkpoint-63000")

loading configuration file /content/Model/Models-Train-14/checkpoint-63000/config.json
Model config ViTConfig {
  "_name_or_path": "/content/Model/Models-Train-14/checkpoint-119000_f",
  "architectures": [
    "ViTForImageClassification"
  ],
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "id2label": {
    "0": "apple_pie",
    "1": "baby_back_ribs",
    "2": "baklava",
    "3": "beef_carpaccio",
    "4": "beef_tartare",
    "5": "beet_salad",
    "6": "beignets",
    "7": "bibimbap",
    "8": "bread_pudding",
    "9": "breakfast_burrito",
    "10": "bruschetta",
    "11": "caesar_salad",
    "12": "cannoli",
    "13": "caprese_salad",
    "14": "carrot_cake",
    "15": "ceviche",
    "16": "cheese_plate",
    "17": "cheesecake",
    "18": "chicken_curry",
    "19": "chicken_quesadilla",
    "20": "chicken_wings",
    "21": "chocolate_cake",
    "22": "chocolate_mousse",
    "23": "churros",
 

# Standard Experiment

In [ ]:
from datasets import load_dataset

ds = load_dataset("imagefolder", data_dir="/content/Dataset(s)/fused-food-101-test", split="train")

In [ ]:
ds

# 100% Missing Modality Experiment

In [ ]:
from datasets import load_dataset

ds = load_dataset("imagefolder", data_dir="/content/Dataset(s)/joint-food-101/test", split="train")

In [ ]:
ds

In [ ]:
# Use _3.png for Filtering out Encoded Images
# Use _4.png for Filtering out Just Images

import os
def filter_funtion(example):
    img = example["image"]
    filename = os.path.basename(img.filename)

    return filename.endswith("_3.png")

In [ ]:
ds = ds.filter(filter_funtion)

In [ ]:
ds

In [ ]:
for i in range(0,150):
    sample = ds[i]
    img = sample["image"]
    filename = os.path.basename(img.filename)
    print(f"File Name: {filename}")

# Partial Missing Modality Experiment

In [4]:
from datasets import load_dataset
from datasets import concatenate_datasets

In [5]:
# Load the Fused Dataset
ds_f = load_dataset("imagefolder", data_dir="/content/Dataset(s)/fused-food-101-test", split="train")

Resolving data files:   0%|          | 0/22716 [00:00<?, ?it/s]

In [6]:
ds_f

Dataset({
    features: ['image', 'label'],
    num_rows: 22716
})

In [7]:
# Desired Percentage = Total - Missing
import math
desired_percentage = 0.5

In [8]:
from collections import Counter

class_counts = Counter(ds_f['label'])
print(class_counts)

Counter({42: 238, 62: 238, 95: 237, 82: 236, 84: 236, 76: 235, 100: 235, 0: 234, 47: 234, 55: 234, 57: 234, 65: 234, 72: 234, 17: 233, 45: 233, 53: 233, 59: 233, 96: 233, 21: 232, 31: 232, 52: 232, 58: 232, 81: 232, 98: 232, 10: 231, 14: 231, 33: 231, 63: 231, 67: 231, 12: 230, 23: 230, 70: 230, 92: 230, 30: 229, 44: 229, 49: 229, 93: 229, 29: 228, 32: 228, 36: 228, 51: 228, 71: 228, 87: 228, 89: 228, 11: 227, 15: 227, 64: 227, 77: 227, 83: 227, 2: 226, 8: 226, 86: 226, 4: 225, 7: 225, 27: 225, 50: 225, 73: 225, 5: 224, 6: 224, 16: 224, 22: 224, 24: 224, 28: 224, 46: 224, 66: 224, 80: 224, 41: 223, 48: 223, 56: 223, 69: 223, 94: 223, 97: 223, 3: 222, 18: 222, 1: 221, 26: 221, 13: 220, 19: 220, 25: 220, 79: 220, 20: 219, 88: 219, 75: 218, 91: 218, 68: 217, 39: 216, 61: 215, 85: 215, 9: 214, 34: 213, 90: 213, 37: 212, 40: 212, 43: 212, 74: 211, 99: 211, 78: 210, 38: 208, 54: 203, 35: 202, 60: 197})


In [9]:
selected_indices = {label: [] for label in class_counts.keys()}
print(selected_indices)

{0: [], 1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: [], 9: [], 10: [], 11: [], 12: [], 13: [], 14: [], 15: [], 16: [], 17: [], 18: [], 19: [], 20: [], 21: [], 22: [], 23: [], 24: [], 25: [], 26: [], 27: [], 28: [], 29: [], 30: [], 31: [], 32: [], 33: [], 34: [], 35: [], 36: [], 37: [], 38: [], 39: [], 40: [], 41: [], 42: [], 43: [], 44: [], 45: [], 46: [], 47: [], 48: [], 49: [], 50: [], 51: [], 52: [], 53: [], 54: [], 55: [], 56: [], 57: [], 58: [], 59: [], 60: [], 61: [], 62: [], 63: [], 64: [], 65: [], 66: [], 67: [], 68: [], 69: [], 70: [], 71: [], 72: [], 73: [], 74: [], 75: [], 76: [], 77: [], 78: [], 79: [], 80: [], 81: [], 82: [], 83: [], 84: [], 85: [], 86: [], 87: [], 88: [], 89: [], 90: [], 91: [], 92: [], 93: [], 94: [], 95: [], 96: [], 97: [], 98: [], 99: [], 100: []}


In [10]:
# Iterate through the dataset to select indices for each class
for i, label in enumerate(ds_f["label"]):
    if len(selected_indices[label]) < math.ceil(class_counts[label] * desired_percentage):
        selected_indices[label].append(i)

In [11]:
# selected_indices

In [12]:
# Flatten the selected indices list
selected_indices = [idx for indices in selected_indices.values() for idx in indices]

In [13]:
# selected_indices

In [14]:
# Filter the dataset to select the desired samples
fused_ds = ds_f.select(selected_indices)

In [15]:
fused_ds

Dataset({
    features: ['image', 'label'],
    num_rows: 11381
})

In [16]:
# Load the Joint Dataset and Filter out Just the Images
ds_joint = load_dataset("imagefolder", data_dir="/content/Dataset(s)/joint-food-101/test", split="train")

Resolving data files:   0%|          | 0/45432 [00:00<?, ?it/s]

In [17]:
# Use _4.png for Filtering out Just Images

import os
def filter_funtion(example):
    img = example["image"]
    filename = os.path.basename(img.filename)

    return filename.endswith("_4.png")

In [18]:
ds_imgs = ds_joint.filter(filter_funtion)

In [19]:
ds_imgs

Dataset({
    features: ['image', 'label'],
    num_rows: 22716
})

In [20]:
# for i in range(10,100):
#     sample = ds_imgs[i]
#     img = sample["image"]
#     filename = os.path.basename(img.filename)
#     print(f"File Name: {filename}")

In [21]:
all_indices = range(len(ds_imgs))
print(all_indices)

range(0, 22716)


In [22]:
remaining_indices = [idx for idx in all_indices if idx not in selected_indices]

In [23]:
# remaining_indices

In [24]:
missing_ds = ds_imgs.select(remaining_indices)

In [25]:
missing_ds

Dataset({
    features: ['image', 'label'],
    num_rows: 11335
})

In [26]:
# Combine Both Datasets

In [27]:
ds = concatenate_datasets([fused_ds, missing_ds])

In [28]:
ds

Dataset({
    features: ['image', 'label'],
    num_rows: 22716
})

# Pre-Processing and Validation Loop

In [29]:
labels = ds.features["label"]
labels

ClassLabel(names=['apple_pie', 'baby_back_ribs', 'baklava', 'beef_carpaccio', 'beef_tartare', 'beet_salad', 'beignets', 'bibimbap', 'bread_pudding', 'breakfast_burrito', 'bruschetta', 'caesar_salad', 'cannoli', 'caprese_salad', 'carrot_cake', 'ceviche', 'cheese_plate', 'cheesecake', 'chicken_curry', 'chicken_quesadilla', 'chicken_wings', 'chocolate_cake', 'chocolate_mousse', 'churros', 'clam_chowder', 'club_sandwich', 'crab_cakes', 'creme_brulee', 'croque_madame', 'cup_cakes', 'deviled_eggs', 'donuts', 'dumplings', 'edamame', 'eggs_benedict', 'escargots', 'falafel', 'filet_mignon', 'fish_and_chips', 'foie_gras', 'french_fries', 'french_onion_soup', 'french_toast', 'fried_calamari', 'fried_rice', 'frozen_yogurt', 'garlic_bread', 'gnocchi', 'greek_salad', 'grilled_cheese_sandwich', 'grilled_salmon', 'guacamole', 'gyoza', 'hamburger', 'hot_and_sour_soup', 'hot_dog', 'huevos_rancheros', 'hummus', 'ice_cream', 'lasagna', 'lobster_bisque', 'lobster_roll_sandwich', 'macaroni_and_cheese', 'mac

In [30]:
labels.int2str(ds[532]["label"])

'beef_tartare'

In [31]:
def transform(examples):
  inputs = image_processor([img.convert("RGB") for img in examples["image"]], return_tensors="pt")
  inputs["labels"] = examples["label"]

  return inputs

In [32]:
dataset = ds.with_transform(transform)

In [33]:
dataset

Dataset({
    features: ['image', 'label'],
    num_rows: 22716
})

In [34]:
for item in dataset:
  print(item["pixel_values"].shape)
  print(item["labels"])
  break

torch.Size([3, 224, 224])
0


In [35]:
labels = ds.features["label"].names
labels

['apple_pie',
 'baby_back_ribs',
 'baklava',
 'beef_carpaccio',
 'beef_tartare',
 'beet_salad',
 'beignets',
 'bibimbap',
 'bread_pudding',
 'breakfast_burrito',
 'bruschetta',
 'caesar_salad',
 'cannoli',
 'caprese_salad',
 'carrot_cake',
 'ceviche',
 'cheese_plate',
 'cheesecake',
 'chicken_curry',
 'chicken_quesadilla',
 'chicken_wings',
 'chocolate_cake',
 'chocolate_mousse',
 'churros',
 'clam_chowder',
 'club_sandwich',
 'crab_cakes',
 'creme_brulee',
 'croque_madame',
 'cup_cakes',
 'deviled_eggs',
 'donuts',
 'dumplings',
 'edamame',
 'eggs_benedict',
 'escargots',
 'falafel',
 'filet_mignon',
 'fish_and_chips',
 'foie_gras',
 'french_fries',
 'french_onion_soup',
 'french_toast',
 'fried_calamari',
 'fried_rice',
 'frozen_yogurt',
 'garlic_bread',
 'gnocchi',
 'greek_salad',
 'grilled_cheese_sandwich',
 'grilled_salmon',
 'guacamole',
 'gyoza',
 'hamburger',
 'hot_and_sour_soup',
 'hot_dog',
 'huevos_rancheros',
 'hummus',
 'ice_cream',
 'lasagna',
 'lobster_bisque',
 'lobster

In [36]:
import torch

def collate_fn(batch):
  return {
      "pixel_values": torch.stack([x["pixel_values"] for x in batch]),
      "labels": torch.tensor([x["labels"] for x in batch]),
  }

In [37]:
dataset

Dataset({
    features: ['image', 'label'],
    num_rows: 22716
})

In [38]:
from torch.utils.tensorboard import SummaryWriter
from torch.optim import AdamW
from torch.utils.data import DataLoader

In [39]:
batch_size = 32

In [40]:
valid_dataset_loader = DataLoader(dataset, collate_fn=collate_fn, batch_size=batch_size, shuffle=False)

In [41]:
model = model.to(device)
model.eval()
predictions, labels = [], []
valid_loss = 0
for batch in valid_dataset_loader:
    pixel_values = batch["pixel_values"].to(device)
    label_ids = batch["labels"].to(device)

    outputs = model(pixel_values=pixel_values, labels=label_ids)

    loss = outputs.loss
    valid_loss += loss.item()

    logits = outputs.logits.detach().cpu()

    predictions.extend(logits.argmax(dim=-1).tolist())
    labels.extend(label_ids.tolist())

In [42]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report

accuracy = accuracy_score(labels, predictions)
precision = precision_score(labels, predictions, average='weighted')
recall = recall_score(labels, predictions, average='weighted')
f1 = f1_score(labels, predictions, average='weighted')
# AUROC_score = roc_auc_score(labels, predictions)

In [43]:
print("Accuracy: ", accuracy)
print("precision: ", precision)
print("f1_score: ", f1)
# print("AUROC_score: ", AUROC_score)

Accuracy:  0.780815284381053
precision:  0.7933427971079401
f1_score:  0.7816787580689748


In [44]:
print(classification_report(labels, predictions))

              precision    recall  f1-score   support

           0       0.86      0.74      0.79       234
           1       0.80      0.81      0.81       221
           2       0.81      0.92      0.86       226
           3       0.83      0.79      0.81       222
           4       0.72      0.53      0.61       225
           5       0.82      0.88      0.85       224
           6       0.82      0.75      0.78       224
           7       0.83      0.73      0.78       225
           8       0.86      0.72      0.78       226
           9       0.76      0.78      0.77       214
          10       0.85      0.83      0.84       231
          11       0.84      0.92      0.88       227
          12       0.90      0.76      0.82       230
          13       0.89      0.85      0.87       220
          14       0.72      0.77      0.74       231
          15       0.88      0.71      0.79       227
          16       0.61      0.69      0.65       224
          17       0.86    

In [45]:
cm = confusion_matrix(labels, predictions)
print(cm)

[[173   1   3 ...   0   0   5]
 [  0 180   1 ...   0   0   0]
 [  3   0 207 ...   0   0   0]
 ...
 [  0   0   3 ... 183   0   0]
 [  0   0   0 ...   0 156   0]
 [  1   0   0 ...   0   0 196]]
